## Royal British Legion Interactive Veteran Display

This notebook was built to create an interactive display where users could easily generate there own plots for different subsets of veteran data

In [1]:
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import panel as pn
import holoviews as hv

# Bokeh panel extension
pn.extension('bokeh', 'tabulator')

In [2]:
# Load data from excels
age = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_age.xlsx")
disability = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_disability.xlsx")
economic_act = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_economic_activity.xlsx")
health = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_general_health.xlsx")
occupation = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_occupation.xlsx")

# Load geographical data
geojson_path = r"C:\Users\tom_r\Desktop\RBL_interview\datasets\Local_Authority_Districts_May_2021_UK_BFC_2022_-815591597668846243.geojson"
gdf = gpd.read_file(geojson_path)

# simplifying to reduce processing time
gdf['geometry'] = gdf['geometry'].simplify(tolerance=100, preserve_topology=True)

In [14]:
disability.head(10)

,Lower tier local authorities Code,Lower tier local authorities,UK armed forces veteran indicator (5 categories) Code,UK armed forces veteran indicator (5 categories),Disability - Equality act disabled (4 categories) Code,Disability - Equality act disabled (4 categories),Observation
0,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,0
1,E06000001,Hartlepool,-8,Does not apply,1,Disabled under the Equality Act: Day-to-day ac...,568
2,E06000001,Hartlepool,-8,Does not apply,2,Disabled under the Equality Act: Day-to-day ac...,828
3,E06000001,Hartlepool,-8,Does not apply,3,Not disabled under the Equality Act,16256
4,E06000001,Hartlepool,1,Previously served in the UK regular armed forces,-8,Does not apply,0
5,E06000001,Hartlepool,1,Previously served in the UK regular armed forces,1,Disabled under the Equality Act: Day-to-day ac...,530
6,E06000001,Hartlepool,1,Previously served in the UK regular armed forces,2,Disabled under the Equality Act: Day-to-day ac...,530
7,E06000001,Hartlepool,1,Previously served in the UK regular armed forces,3,Not disabled under the Equality Act,1971
8,E06000001,Hartlepool,2,Previously served in UK reserve armed forces,-8,Does not apply,0
9,E06000001,Hartlepool,2,Previously served in UK reserve armed forces,1,Disabled under the Equality Act: Day-to-day ac...,111


In [3]:
disability['UK armed forces veteran indicator (5 categories) Code'].unique()

array([-8,  1,  2,  3,  4], dtype=int64)

In [4]:
# Prepare the disability data
total_number_vets = disability[
    (disability['UK armed forces veteran indicator (5 categories) Code'] == 1) | 
    (disability['UK armed forces veteran indicator (5 categories) Code'] == 2)
]
veterans_sum = total_number_vets.groupby('Lower tier local authorities')['Observation'].sum().reset_index()
veterans_sum.columns = ['Lower tier local authorities', 'Total Veterans']
veterans_sum.rename(columns={'Lower tier local authorities': 'LAD21NM'}, inplace=True)

# merge with geographical data
merged_gdf = gdf.merge(veterans_sum, on='LAD21NM', how='left')
merged_gdf['Total Veterans'] = merged_gdf['Total Veterans'].fillna(0).astype(int)
merged_gdf.rename(columns={'LAD21NM': 'Area'}, inplace=True)

# convert coodinate ref system
gdf = merged_gdf.to_crs(epsg=4326)

In [15]:
merged_gdf.head()

,OBJECTID,LAD21CD,Area,LAD21NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry,Total Veterans
0,1,E06000001,Hartlepool,,447160,531474,-1.27018,54.67614,6497151b-869e-454f-b943-ef2e32473b9c,"MULTIPOLYGON (((450140.312 525924.854, 450150....",3684
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.54467,30975abf-b3f8-453e-9539-4a1578310fb9,"MULTIPOLYGON (((451752.698 520561.900, 451649....",4424
2,3,E06000003,Redcar and Cleveland,,464361,519597,-1.00608,54.56752,58c003e1-48b1-4b5a-a412-5dcec0794989,"MULTIPOLYGON (((454907.003 526643.599, 454882....",6702
3,4,E06000004,Stockton-on-Tees,,444940,518183,-1.30664,54.55691,8827e88c-dee1-4222-a0ed-8cc299f622a0,"MULTIPOLYGON (((450121.840 525888.860, 450118....",7931
4,5,E06000005,Darlington,,428029,515648,-1.56835,54.53534,1f43640c-1690-43a5-8ad4-925199b2e787,"POLYGON ((423496.602 524724.299, 423896.100 52...",5132


In [5]:
# filters dataset by selected areas, veteran indicators, and a given category
def filter_data(dataset, selected_names, selected_indicators, by_column, selected_by_values):
    data = dataset[dataset['Lower tier local authorities'].isin(selected_names)]
    if data.empty:
        print(f"No data available for areas: {selected_names}")
        return pd.DataFrame()
    filtered_data = data[
        (data['UK armed forces veteran indicator (5 categories)'].isin(selected_indicators)) &
        (data[by_column].isin(selected_by_values)) &
        (data['UK armed forces veteran indicator (5 categories)'] != -8) & 
        (data['UK armed forces veteran indicator (5 categories)'] != 4)
    ]
    grouped_data = filtered_data.groupby(['UK armed forces veteran indicator (5 categories)', by_column])['Observation'].sum().unstack().fillna(0).reset_index()
    long_format_data = grouped_data.melt(id_vars=['UK armed forces veteran indicator (5 categories)'], var_name=by_column, value_name='Observation')
    return long_format_data

In [6]:
# Call the filter_date() function for each parameter and generates a bar plot

def generate_plots(selected_names, selected_indicators, 
                   disability_by, health_by, age_by, economic_by, occupation_by,
                   disability_by_values, health_by_values, age_by_values, economic_by_values, occupation_by_values):
    
    if not selected_names:
        selected_names = list(gdf['Area'])
    if not selected_indicators:
        selected_indicators = list(disability['UK armed forces veteran indicator (5 categories)'].unique())
    
    # dynamic title
    title_suffix = f"for {', '.join(selected_names[:3])}..." if len(selected_names) > 3 else f"for {', '.join(selected_names)}"

    # Disability Plot
    disability_data = filter_data(disability, selected_names, selected_indicators, disability_by, disability_by_values)
    if not disability_data.empty:
        disability_plot = disability_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=disability_by,
            stacked=False,
            rot=45,
            title=f'Disability by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        disability_plot = pn.pane.Markdown("No data available for the selected areas in the disability dataset.")
    
    # Health Plot
    health_data = filter_data(health, selected_names, selected_indicators, health_by, health_by_values)
    if not health_data.empty:
        health_plot = health_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=health_by,
            stacked=False,
            rot=45,
            title=f'Health by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        health_plot = pn.pane.Markdown("No data available for the selected areas in the health dataset.")
    
    # Age Plot
    age_data = filter_data(age, selected_names, selected_indicators, age_by, age_by_values)
    if not age_data.empty:
        age_plot = age_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=age_by,
            stacked=False,
            rot=45,
            title=f'Age by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        age_plot = pn.pane.Markdown("No data available for the selected areas in the age dataset.")
    
    # Economic Activity Plot
    economic_data = filter_data(economic_act, selected_names, selected_indicators, economic_by, economic_by_values)
    if not economic_data.empty:
        economic_plot = economic_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=economic_by,
            stacked=False,
            rot=45,
            title=f'Economic Activity by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        economic_plot = pn.pane.Markdown("No data available for the selected areas in the economic activity dataset.")
    
    # Occupation Plot
    occupation_data = filter_data(occupation, selected_names, selected_indicators, occupation_by, occupation_by_values)
    if not occupation_data.empty:
        occupation_plot = occupation_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=occupation_by,
            stacked=False,
            rot=45,
            title=f'Occupation by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        occupation_plot = pn.pane.Markdown("No data available for the selected areas in the occupation dataset.")
    
    return pn.Tabs(
        ("Disability", disability_plot),
        ("Health", health_plot),
        ("Age", age_plot),
        ("Economic Activity", economic_plot),
        ("Occupation", occupation_plot)
    )

In [7]:

# widgets for selecting areas and veteran indicators
area_select = pn.widgets.MultiSelect(
    name='Select Areas',
    options=list(gdf['Area'].unique()),
    value=list(gdf['Area'].unique()),  # Select all areas by default
    size=10
)

veteran_indicator_options = list(disability['UK armed forces veteran indicator (5 categories)'].unique())
veteran_indicator_select = pn.widgets.MultiSelect(
    name='Select Veteran Indicator(s)',
    options=veteran_indicator_options,
    value=veteran_indicator_options, 
    size=10
)


In [8]:
def get_unique_values(df, column):
    return sorted(df[column].unique().tolist())

In [9]:
disability_by_values_select = pn.widgets.MultiSelect(
    name='Select Disability Values',
    options=get_unique_values(disability, 'Disability - Equality act disabled (4 categories)'),
    value=get_unique_values(disability, 'Disability - Equality act disabled (4 categories)'),
    size=10
)

health_by_values_select = pn.widgets.MultiSelect(
    name='Select Health Values',
    options=get_unique_values(health, 'General health (3 categories)'),
    value=get_unique_values(health, 'General health (3 categories)'),
    size=10
)

age_by_values_select = pn.widgets.MultiSelect(
    name='Select Age Values',
    options=get_unique_values(age, 'Age (B) (14 categories)'),
    value=get_unique_values(age, 'Age (B) (14 categories)'),
    size=10
)

economic_by_values_select = pn.widgets.MultiSelect(
    name='Select Economic Activity Values',
    options=get_unique_values(economic_act, 'Economic activity status (4 categories)'),
    value=get_unique_values(economic_act, 'Economic activity status (4 categories)'),
    size=10
)

occupation_by_values_select = pn.widgets.MultiSelect(
    name='Select Occupation Values',
    options=get_unique_values(occupation, 'Occupation (current) (10 categories)'),
    value=get_unique_values(occupation, 'Occupation (current) (10 categories)'),
    size=10
)


In [10]:

# Bind the plot generation
plot_tabs = pn.bind(
    generate_plots,
    area_select,
    veteran_indicator_select,
    'Disability - Equality act disabled (4 categories)',
    'General health (3 categories)',
    'Age (B) (14 categories)',
    'Economic activity status (4 categories)',
    'Occupation (current) (10 categories)',
    disability_by_values_select,
    health_by_values_select,
    age_by_values_select,
    economic_by_values_select,
    occupation_by_values_select
)

# Create interactive map 
initial_map_plot = gdf.hvplot.polygons(
    geo=True,
    tiles='CartoLight',
    frame_height=600,
    frame_width=450,
    color='Total Veterans',
    cmap='Viridis',
    line_color='black',
    hover_cols=['Area', 'Total Veterans'],
    title="Total Veterans in UK Areas As Estimated by Census Data",
    tools=['tap']
)

# stream for tap evenmts
tap_stream = hv.streams.Tap(source=initial_map_plot, x=None, y=None)

# update the map depending on mouse click
def update_selection(x, y):
    if x is None or y is None:
        return area_select.value 
    nearest_point = gdf.sindex.nearest((x, y, x, y), return_distance=True)
    if nearest_point[1][0] > 0.01:  
        return area_select.value 
    selected_area = gdf.iloc[nearest_point[0]]['Area']
    return [selected_area]

tap_stream.add_subscriber(lambda x, y: area_select.value.update(update_selection(x, y)))

# Panel for the map
map_panel = pn.panel(initial_map_plot)

sidebar = pn.Column(
    pn.Row(
        pn.pane.PNG('Royal_British_Legion_logo.png', width=300, margin=(0, 3, 0, 0)),
        pn.Column("## Select Area(s)", area_select, sizing_mode='stretch_width')
    ),
    map_panel
)


In [11]:

# Dashboard layout with veteran indicator selector on the right 
dashboard = pn.template.FastListTemplate(
    title='Interactive Dashboard',
    sidebar_width=750,
    sidebar=sidebar,
    main=[
        pn.Row(
            plot_tabs,
            pn.Column(
                "## Select Veteran Indicator(s)",
                veteran_indicator_select,
                "## Select Disability Values",
                pn.Row(disability_by_values_select),
                "## Select Health Values",
                pn.Row(health_by_values_select),
                "## Select Age Values",
                pn.Row(age_by_values_select),
                "## Select Economic Activity Values",
                pn.Row(economic_by_values_select),
                "## Select Occupation Values",
                pn.Row(occupation_by_values_select),
                width=350, sizing_mode='stretch_height'
            )
        )
    ],
    accent_base_color="#004b87",  # RBL Blue
)

In [16]:
# Serve the dashboard in a new browser window
pn.serve(dashboard, start=True, show=True)

Launching server at http://localhost:63706
